In [1]:
import os
import random
import numpy as np
import cv2
import tensorflow as tf
from keras import layers, Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split



In [2]:


dataset_path = "C:\\Users\\14384\\Desktop\\Project\\resized_data"

# list of classes
classes = ['bambalouni', 'brik', 'chakchouka', 'couscous', 'kafteji', 'lablabi', 'makroudh', 'marqa', 'mloukhia', 'tajine']


def tunisianFoodCNN(input_shape, num_classes):
    # Create a Sequential model
    model = Sequential()

    # Add convolutional layers
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    # Flatten the feature maps
    model.add(Flatten())

    # Add dense layers for classification
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))  # Output layer with num_classes units

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Example usage:
input_shape = (224, 224, 3)
num_classes = 10
custom_cnn_model = tunisianFoodCNN(input_shape, num_classes)

# Check the model summary
custom_cnn_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [3]:
# A function to load and preprocess images
def load_and_preprocess_image(image_path, target_size=(224, 224),dtype=np.float32):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = image.astype(dtype) / 255.0  # Normalize to [0, 1]
    return image



In [4]:
# Initialize empty lists for labels and file paths
all_data = []
all_labels = []


# Function to get class index based on folder name
def get_class_index(folder_name):
    return classes.index(folder_name)

# Iterate through all image files in the data directory
for class_folder in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_folder)
    class_index = get_class_index(class_folder)
    
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        all_labels.append(to_categorical(class_index, num_classes=10))
        all_data.append(load_and_preprocess_image(image_path))

# Split your data into training, validation, and test sets as needed
train_data, test_data, train_labels, test_labels = train_test_split(all_data, all_labels, test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.1, random_state=42)

# Convert the label lists to numpy arrays
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)
test_labels = np.array(test_labels)


 one-hot encoding technique to represent the labels for each image. In one-hot encoding, each class is represented by a vector of zeros, except for the index of the class, which is set to 1. For example, the label for the image of a bambalouni would be a vector of zeros, except for the index of the bambalouni class, which would be set to 1.

Build and Compile Model

In [5]:
# Define input_shape and num_classes
input_shape = (224, 224, 3)
num_classes = 10

# Create the custom CNN model using the function
model = tunisianFoodCNN(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up model checkpoint to save the best weights
checkpoint = ModelCheckpoint('custom_cnn_weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)



**Train the model**

In [6]:
batch_size = 16
epochs = 30

history = model.fit(np.array(train_data), train_labels, batch_size=batch_size,
                    epochs=epochs, validation_data=(np.array(val_data), val_labels),
                    callbacks=[checkpoint])

# Evaluate the model on the test set
test_loss = model.evaluate(np.array(test_data), np.array(test_labels))

# Save the model
model.save('Cnn_model_tunisianFood.h5')


Epoch 1/30
673/673 [==============================] - 432s 639ms/step - loss: 2.2061 - accuracy: 0.1801 - val_loss: 2.0604 - val_accuracy: 0.2598
Epoch 2/30
673/673 [==============================] - 373s 555ms/step - loss: 1.8312 - accuracy: 0.3467 - val_loss: 1.9121 - val_accuracy: 0.2949
Epoch 3/30
673/673 [==============================] - 376s 559ms/step - loss: 1.3035 - accuracy: 0.5467 - val_loss: 2.1508 - val_accuracy: 0.3091
Epoch 4/30
673/673 [==============================] - 406s 603ms/step - loss: 0.6118 - accuracy: 0.7967 - val_loss: 2.7224 - val_accuracy: 0.3467
Epoch 5/30
673/673 [==============================] - 399s 593ms/step - loss: 0.2073 - accuracy: 0.9398 - val_loss: 4.0830 - val_accuracy: 0.3450
Epoch 6/30
673/673 [==============================] - 351s 522ms/step - loss: 0.1180 - accuracy: 0.9692 - val_loss: 4.9623 - val_accuracy: 0.3367
Epoch 7/30
673/673 [==============================] - 358s 532ms/step - loss: 0.0811 - accuracy: 0.9800 - val_loss: 5.1889 -

c:\Users\14384\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**Predictions**

In [7]:
def predict_class(image_path):
    image = load_and_preprocess_image(image_path)
    predictions = model.predict(np.array([image]))
    class_index = np.argmax(predictions)
    class_name = classes[class_index]  # Assuming you have defined 'classes' as a list of class names
    return class_name

REAL CLASS : couscous

In [8]:
image_path= 'C:\\Users\\14384\\Desktop\\Project\\Predict\\testingphoto1REALkosksi.jpg'
predicted_class = predict_class(image_path)
print("Predicted Class:", predicted_class)

1/1 [==============================] - 0s 188ms/step
Predicted Class: brik


REAL CLASS: lablabi

In [9]:
image_path1 = 'C:\\Users\\14384\\Desktop\\Project\\Predict\\testingphoto2REAL-lablebi.jpg'

predicted_class1 = predict_class(image_path1)
print("Predicted Class:", predicted_class1)

1/1 [==============================] - 0s 31ms/step
Predicted Class: bambalouni


In [10]:
image_path1 = 'C:\\Users\\14384\\Desktop\\Project\\Predict\\testingphoto3REALtajine.jpg'

predicted_class1 = predict_class(image_path1)
print("Predicted Class:", predicted_class1)

1/1 [==============================] - 0s 32ms/step
Predicted Class: mloukhia


In [11]:
image_path1 = 'C:\\Users\\14384\\Desktop\\Project\\Predict\\testingphoto4REALmarga.jpg'

predicted_class1 = predict_class(image_path1)
print("Predicted Class:", predicted_class1)

1/1 [==============================] - 0s 31ms/step
Predicted Class: mloukhia


In [ ]:
# # Make predictions on new images
# def predict(image_path):
#     image = load_and_preprocess_image(image_path)
#     predictions = model.predict(np.array([image]))
#     return predictions
# image_path1 = 'C:\Users\14384\Desktop\Project\Predict\testingphoto1REALkosksi.jpg'
# predictions1 = predict(image_path)

# # class probabilities
# print(predictions1)
